# Handwritten Digit Classification using Neural Network

### Problem Description

Given a dataset "ex4data1.mat", consisting of 5000 training examples of handwritten digits. Each training example is a 20 * 20 pixel grayscale image of the digit. Each pixel is represented by a floating number corresponding to intensity at the location. Implement a Neural Network (feedforward and backpropagation algorithm) for predicting the handwritten digits in the set.

Feature set X = 5000 * 400 matrix (each row is training example for handwritten digit image) 
Target y = 5000 dimension vector with labels for each training set.

Dataset: All taken from "Machine Learning by Stanford University Coursera".

In [1]:
import scipy.io as sio
import numpy as np

### Loading the dataset

In [2]:
data = sio.loadmat('ex4data1.mat')

In [3]:
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [4]:
# Feature, target separation
X = data.get('X')
y = data.get('y')

In [5]:
X.shape, y.shape

((5000, 400), (5000, 1))

### Initialization

Assume that the neural network model has 3 layers. Layer-1 is input layer consisting of 400 units (size of X), layer-2 is the hidden layer consisting of 25 units and layer-3 is the output layer with 10 units (number of classes)

In [6]:
num_labels = 10 # we need to classify 10 labels
hidden_units = 25 
input_layer = 400
regParam = 1 # lambda for regularization

Since we are using 10 units in output layer, we would need an encoder for each class. Let's use the One-hot encoder from sklearn to generate a vector for each class. Out of 10 classes, One-hot encoding turns a class label i into a vector of length 10 where the index i = 1 while the rest are zero.

In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
encoder = OneHotEncoder(sparse=False)
y_vec = encoder.fit_transform(y)
y_vec.shape

(5000, 10)

In [9]:
y_vec

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

Lets generate random weights for this model. We could use np.random package to generate theta values.

In [10]:
theta_vec = (np.random.random(size = hidden_units * (input_layer + 1) + num_labels * (hidden_units + 1)) - 0.5)* 0.25

### Basic functions 

This section consists of basic functions needed for implementation of Neural Networks.  
sigmoid() : computes the logistic function  
feedforward() : generates hypothesis of neural network model  
sigmoidGradient() : computes the g'(z) needed in backpropagation algorithm  
backpropagation() : computes the cost, gradient for the given theta

In [11]:
def sigmoid(z):
    return (1.0 / (1 + np.exp(-z)))

In [12]:
def feedforward(X,theta1, theta2):
    # Make sure X, theta1, theta2 are all matrices
    m = X.shape[0] # 5000
    a1 = np.insert(X, 0, values = np.ones(m), axis = 1) # add bias
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values = np.ones(m), axis = 1) # add bias
    z3 = a2 * theta2.T
    a3 = sigmoid(z3)
    
    return [a1,z2,a2,z3,a3] # hypothesis

In [13]:
def sigmoidGradient(z):
    # irrespective of z (vector, scalar, matrix), compute the sigmoid gradient of z
    return (np.multiply(sigmoid(z), (1 - sigmoid(z))))

In [14]:
def backpropagation(theta_vec, num_labels, hidden_units, input_layer, X, y, regParam):
    #theta1 = np.matrix(theta_vec[0 : hidden_units * (input_layer + 1)].reshape(hidden_units, input_layer + 1)) # 25 * 401
    #theta2 = np.matrix(theta_vec[hidden_units * (input_layer + 1), :].reshape(num_labels, hidden_units + 1)) # 10 * 26
    
    theta1 = np.matrix(theta_vec[0: 10025].reshape(25, 401))
    theta2 = np.matrix(theta_vec[10025:].reshape(10,26))
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    feed_list = feedforward(X, theta1, theta2)
    a1 = feed_list[0]
    z2 = feed_list[1]
    a2 = feed_list[2]
    hyp = feed_list[4]
    
    # Compute cost
    J = 0 #initialize
    
    for i in range(m):
        term1 = np.multiply(-y[i,:], np.log(hyp[i,:]))
        term2 = np.multiply((1 - y[i,:]), np.log(1 - hyp[i,:]))
        J += np.sum(term1 - term2)
    
    # adding regularization term
    regTerm = np.sum(np.power(theta1[:,1:],2)) + np.sum(np.power(theta2[:,1:],2))
    J = J/m + (regParam * regTerm / (2.0 * m))
    
    # back propagation algorithm 
    delta1 = np.zeros(theta1.shape) # 25 * 401
    delta2 = np.zeros(theta2.shape) # 10 * 26
    
    for t in range(m):
        a1t = a1[t,:] # 1, 401
        z2t = z2[t,:] # 1, 25
        a2t = a2[t,:] # 1, 26
        a3t = hyp[t,:] # 1, 10
        yt = y[t,:] # 1, 10
        
        del3 = a3t - yt # 1, 10
        z2t = np.insert(z2t, 0, values=np.ones(1)) # 1, 26
        del2 = np.multiply((theta2.T * del3.T).T, sigmoidGradient(z2t)) # 1, 26 
        delta1 = delta1 + (del2[:,1:]).T * a1t # 25, 401
        delta2 = delta2 + del3.T * a2t # 10, 26
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * regParam) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * regParam) / m
       
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
        
    return [J, grad]

### Training the Neural Network

Using the basic functions (defined above), lets train the neural network with some random weights "theta_vec" and obtain the cost, gradient. This is fed to the advanced optimization algorithm to minimize the cost for chosen theta.

In [15]:
J, grad = backpropagation(theta_vec, 400, 25, 10, X, y_vec, regParam)

In [16]:
J, grad.shape

(7.3257574115134, (10285,))

In [17]:
#Lets minimize the cost using the optimization function

In [18]:
from scipy.optimize import minimize

In [19]:
fmin = minimize(fun=backpropagation, x0=theta_vec, args=(num_labels, hidden_units, input_layer, X, y_vec, regParam), 
                method='TNC', jac=True, options={'maxiter': 250})

In [20]:
fmin

     fun: 0.33943629243809315
     jac: array([ 1.46533056e-04,  5.14717127e-07, -5.20537949e-07, ...,
       -5.09097449e-05, -3.85048088e-06, -8.45565480e-05])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 19
  status: 3
 success: False
       x: array([-1.0011272 ,  0.00257359, -0.00260269, ...,  0.49431216,
        0.03453273,  1.46771324])

### Prediction and Evaluation

Using the weights obtained from optimized function, let's predict the value for a randomly selected input and evaluate the accuracy of prediction

In [21]:
theta1 = np.matrix(np.reshape(fmin.x[0 : hidden_units * (input_layer + 1)], (25, 401)))
theta2 = np.matrix(np.reshape(fmin.x[hidden_units * (input_layer + 1) : ], (10, 26)))

feed_list = feedforward(X, theta1, theta2)
prediction = np.array(np.argmax(feed_list[4], axis = 1) + 1) # feed_list[4] = hypothesis

In [22]:
prediction

array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]])

In [23]:
# Evaluation

In [24]:
correct = [1 if a == b else 0 for (a, b) in zip(prediction, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {} %'.format(accuracy * 100))

accuracy = 99.28 %


#### Thats it!! We have succesfully implemented a feed-forward neural network with backpropagation and used it to classify images of handwritten digits.